In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5ff17c6f54fd23191499c7017d6366538fcc35b61d0225390f78827c662577ddcc8d5d64ddf58c68c56d6ad68e5721930a5b7a5803b0bee173'

In [2]:
import aocd
from aocd.models import Puzzle
day = 19
year = 2023
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [42]:
import yaml
from ast import literal_eval
import operator as op

In [6]:
test_data = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""
data_test = test_data.splitlines()

In [4]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(714,
 ['cqh{m>1849:R,A}',
  'zs{a>1909:ql,x<1340:kxd,m>3378:kt,kbt}',
  'sp{a<665:R,s>1356:A,A}',
  'kf{a>1202:rkv,x>2756:A,jnv}',
  'kck{s<1337:pgl,R}',
  'tnp{x<3631:mm,x<3771:qbx,a>1403:kjs,lxf}',
  'vnj{a>2884:fm,a>2424:gg,mgv}',
  'zxv{s<436:R,a>261:A,R}',
  'pjx{s<1500:A,x>2599:R,vg}',
  'pc{s<2563:A,qx}'])

In [142]:
def parse_rules(rules):
    res = {}
    for line in rules.splitlines():
        idx = line.find('{')
        res[line[:idx]] = line[idx+1:-1].split(',')
    return res

def parse_parts(parts):
    res = []
    for line in parts.splitlines():
        res.append(yaml.safe_load(line.replace('=', ': ')))
    return res

def eval_rule(rule, part):
    if isinstance(rule, bool):
        return rule
    fmap = {
        '>': op.gt,
        '<': op.lt
    }
    if ':' not in rule:
        return rule
    rule, key = rule.split(':')
    attr = rule[0]
    func = fmap[rule[1]]
    val = int(rule[2:])
    if func(part[attr], val):
        return key
    

def sort_part(part, key, rules):
    # print(key)
    for rule in rules[key]:
        res = eval_rule(rule, part)
        match res:
            case None:
                continue
            case str():
                return sort_part(part, res, rules)
            case bool():
                return res
        

In [200]:
# rules, parts = test_data.split('\n\n')
rules, parts = puzzle.input_data.split('\n\n')


parts = parse_parts(parts)
rules = parse_rules(rules)
rules['A'] = ['A']
rules['R'] = ['R']


In [144]:
res = sum([sum(part.values()) for part in parts if sort_part(part, 'in', rules)])
res

397134

In [92]:
puzzle.answer_a = res

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [93]:
# Part 2

In [267]:
def eval_rule_b(rule, part):
    if rule == 'A':
        return 'A', part, None
    elif rule == 'R':
        return 'R', None, None
    # if isinstance(rule, bool):
    #     if rule:
    #         return rule, part, None
    #     else:
    #         return rule, None, None
    
    fmap = {
        '>': op.gt,
        '<': op.lt
    }
    if ':' not in rule:
        return rule, part, None
    
    rule, key = rule.split(':')
    
    attr = rule[0]
    func = fmap[rule[1]]
    val = int(rule[2:])
    
    part1 = part.copy()
    part2 = part.copy()
    
    inr1, inr2 = func(part[attr], val)
    # print(inr1, inr2)
    part1[attr] = inr1
    part2[attr] = inr2
    
    return key, part1, part2
    

def sort_part_b(part, key, rules):
    # print(key)
    ans = []
    for rule in rules[key]:
        if rule == 'A':
            ans += [part]
            break
        elif rule == 'R':
            break
        nxt_rule, part1, part = eval_rule_b(rule, part)
        ans += sort_part_b(part1, nxt_rule, rules)
        if part == None:
            break
    return ans
            
class intrange(tuple):
    def __lt__(self, other):
        if self[1] < other:
            return self, None
        if self[0] >= other:
            return None, self
        return intrange([self[0], other-1]), intrange([other, self[1]])
    def __gt__(self, other):
        if self[0] > other:
            return self, None
        if self[1] <= other:
            return None, self
        return intrange([other+1, self[1]]), intrange([self[0], other])
    def __add__(self, other):
        if isinstance(other, intrange):
            return (self[1]-self[0]+1) * (other[1]-other[0]+1)
        elif isinstance(other, int):
            return (self[1]-self[0]+1) * other
    def __radd__(self, other):
        if isinstance(other, intrange):
            return (self[1]-self[0]+1) * (other[1]-other[0]+1)
        elif isinstance(other, int):
            if other == 0:
                return (self[1]-self[0]+1)
            return (self[1]-self[0]+1) * other

In [268]:
# rules, parts = test_data.split('\n\n')
rules, parts = puzzle.input_data.split('\n\n')


parts = parse_parts(parts)
rules = parse_rules(rules)
rules['A'] = ['A']
rules['R'] = ['R']

In [269]:
part = {
    'x': intrange([1, 4000]),
    'm': intrange([1, 4000]),
    'a': intrange([1, 4000]),
    's': intrange([1, 4000]),
}

In [272]:
accepted = sort_part_b(part, 'in', rules)

In [273]:
res = sum([sum(p.values()) for p in accepted])
res

127517902575337

In [264]:
puzzle.answer_b = res

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 19! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
